In [1]:
!pip3 install tensorflow==2.2.0

In [2]:
from glob import glob
import os 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.notebook import tqdm
import random
def create_dir(base,ext):
    '''
        creates and external dir in base
        args:
            base   =    the main path
            ext    =    the external path
    '''
    _path=os.path.join(base,ext)
    if not os.path.exists(_path):
        os.mkdir(_path)
    return _path

# dataset dir
ds_path  =   create_dir(os.getcwd(),'OcrDTF')
tf_train =   create_dir(ds_path,'Train')
tf_eval  =   create_dir(ds_path,'Eval')
tf_test  =   create_dir(ds_path,'Test')

# image dir
IDEN_START=172
BasicFinalDatabase_path   =    os.path.join(os.getcwd(),'BasicFinalDatabase')
train_path                =    os.path.join(BasicFinalDatabase_path,'Train')
test_path                 =    os.path.join(BasicFinalDatabase_path,'Test')

In [3]:
train_img_paths  =  [img_path for img_path in tqdm(glob(os.path.join(train_path,'*/*.bmp')))]
test_img_paths   =  [img_path for img_path in tqdm(glob(os.path.join(test_path,'*/*.bmp')))]

nb_train=int(len(train_img_paths)*0.8)
random.shuffle(train_img_paths)

eval_img_paths  =  train_img_paths[nb_train:]
train_img_paths =  train_img_paths[:nb_train]

def get_label(img_path):
    '''
        get label from data path
    '''
    _base  =  os.path.dirname(img_path)
    _label =  os.path.basename(_base)
    _label =  int(_label) - IDEN_START
    return _label

print(len(train_img_paths),len(eval_img_paths),len(test_img_paths))


9600 2400 3000


In [4]:
import tensorflow as tf
from PIL import Image

print(tf.__version__)

DATA_NUM=512
IMG_DIM =128

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_feature(value):
      return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _float_feature(value):
      return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))


def to_tfrecord(img_paths,tf_mode,r_num):
    '''
      Creates tfrecords from Provided Image Paths
    '''
    tfrecord_name='{}.tfrecord'.format(r_num)
    tfrecord_path=os.path.join(tf_mode,tfrecord_name) 
    
    with tf.io.TFRecordWriter(tfrecord_path) as writer:    
        
        for img_path in tqdm(img_paths):
            
            #label
            label=get_label(img_path)
            
            # image data
            img = Image.open(img_path).convert('RGB')
            img = img.resize((IMG_DIM,IMG_DIM))
            img.save( 'test.png', 'png')
            
            with(open('test.png','rb')) as fid:
                image_png_bytes=fid.read()
            
            # feature desc
            data ={ 'image':_bytes_feature(image_png_bytes),
                    'label':_int64_feature(label)
            }
            
            features=tf.train.Features(feature=data)
            example= tf.train.Example(features=features)
            serialized=example.SerializeToString()
            writer.write(serialized)  
            
def create_df(img_paths,tf_mode):
    '''
        tf record wrapper
    '''
    for idx in range(0,len(img_paths),DATA_NUM):
        
        _paths=img_paths[idx:idx+DATA_NUM]
        
        to_tfrecord(_paths,tf_mode,idx//DATA_NUM)

2.2.0


In [5]:
create_df(train_img_paths,tf_train)
create_df(eval_img_paths,tf_eval)
create_df(test_img_paths,tf_test)

In [6]:
os.remove('test.png')